# Bird Object Detection - SageMaker Pipelines

Now that we have trained models and deployed them with SageMaker, we will look into how we can create parametrizable and reproducible ML pipelines with SageMaker Pipelines.

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sess = sagemaker.Session()
# this will create a 'default' sagemaker bucket if it doesn't exist (sagemaker-region-accountid)
bucket = sess.default_bucket()

role = get_execution_role()

prefix = "DEMO-ObjectDetection-birds"
TRAIN_REC_FILE = "birds_ssd_sample_train.rec"
VAL_REC_FILE = "birds_ssd_sample_val.rec"
TRAIN_LST_FILE = "birds_ssd_sample_train.lst"
VAL_LST_FILE = "birds_ssd_sample_val.lst"

# Upload the RecordIO files to train and validation channels
train_channel = prefix + "/train"
validation_channel = prefix + "/validation"

sess.upload_data(path="birds_ssd_sample_train.rec", bucket=bucket, key_prefix=train_channel)
sess.upload_data(path="birds_ssd_sample_val.rec", bucket=bucket, key_prefix=validation_channel)

s3_train_data = "s3://{}/{}".format(bucket, train_channel)
s3_validation_data = "s3://{}/{}".format(bucket, validation_channel)

print(s3_train_data)
print(s3_validation_data)

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.mxnet.estimator import MXNet
from sagemaker.processing import FrameworkProcessor

from sagemaker import image_uris

training_image = image_uris.retrieve("mxnet", sess.boto_region_name, instance_type="ml.m5.xlarge", image_scope="training", version="1.8")
print(training_image)

mxnet_processor = FrameworkProcessor(
    MXNet,
    '1.8',
    image_uri=training_image,
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name="mxnet-processor",
    role=role,
    command=["python3"]
)

mxnet_processor.run(
    code='preprocess.py',
    source_dir='src',
    outputs=[ProcessingOutput(source='/opt/ml/processing/output/train'),
        ProcessingOutput(source='/opt/ml/processing/output/validation')]
)